In [1]:
# Import functions
from ultralytics.utils import SETTINGS, Path
from ultralytics import YOLO
from sklearn.model_selection import ParameterGrid
import tqdm
import os
import fiftyone as fo
import fiftyone.zoo as foz
import warnings
import textwrap
import locale

In [2]:
# Layer freezing function
def freeze_layer(trainer):
    model = trainer.model
    num_freeze = 10
    print(f"Freezing {num_freeze} layers")
    freeze = [f'model.{x}.' for x in range(num_freeze)]  # layers to freeze
    for k, v in model.named_parameters():
        v.requires_grad = True  # train all layers
        if any(x in k for x in freeze):
            print(f'freezing {k}')
            v.requires_grad = False
    print(f"{num_freeze} layers are freezed.")

In [19]:
# Define hyperparameters to tune
learning_rates = [0.001]
batch_sizes = [24]
optimizers = ['Adam']
epochs = [50]
freeze_layers = [10]

In [20]:
# Set up search space
search_space = {'learning_rate': learning_rates,
                'batch_size': batch_sizes,
                'optimizer': optimizers,
                'epochs': epochs,
                'freeze_layer': freeze_layers}

In [21]:
# Create parameter grid
parameter_grid = ParameterGrid(search_space)

In [22]:
# How many model combinations
len(parameter_grid)

6

In [6]:
# Instantiate model (yolov8 small)
model = YOLO("yolov8s.pt")

In [7]:
# See layers of model
# for k, v in model.named_parameters():
#     print(k)

In [8]:
# # # Freeze layers of model
# model.add_callback("on_train_start", freeze_layer)

In [9]:
# Run training
# results = model.train(data='/home/jupyter-snarain/datasets/open-images-v7/dataset.yaml', epochs=5, batch=8)
# Train and evaluate the model for each combination of hyperparameters
i = 1
for parameters in parameter_grid:
    # Set hyperparameters for the model
    learning_rate = parameters['learning_rate']
    batch_size = parameters['batch_size']
    optimizer = parameters['optimizer']
    num_epochs = parameters['epochs']
    freeze_layer = parameters['freeze_layer']
    
    # Train and evaluate the model
    results = model.train(data='/home/jupyter-snarain/datasets_2/dataset.yaml', lr0 = learning_rate, batch = batch_size, optimizer = optimizer, epochs = num_epochs, freeze = freeze_layer, 
                          imgsz = 640, patience = 25, save_period = 5, name = f"checkpoint_model_{i}", project = "Hypertuning", device=[0,1,2,3])
    
    # Save the results
    result = {'learning_rate': learning_rate,
              'batch_size': batch_size,
              'optimizer': optimizer,
              'num_epochs': num_epochs,
              'freeze_layer': freeze_layer
             }
    i += 1

New https://pypi.org/project/ultralytics/8.0.208 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.200 🚀 Python-3.10.10 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/home/jupyter-snarain/datasets_2/dataset.yaml, epochs=50, patience=25, batch=8, imgsz=640, save=True, save_period=5, cache=False, device=None, workers=8, project=Hypertuning, name=checkpoint_model_14, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=0.001, profile=False, freeze=0, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=No

In [ ]:
# Resume training - ONLY RUN IF KERNEL DIES
# model = YOLO("/home/jupyter-snarain/scripts/Hypertuning/last.pt")
# model.train(resume=True)

In [ ]:
# def get_instance_id():
#     """Get current instance ID from metadata"""
#     url = "http://169.254.169.254/latest/meta-data/instance-id"
#     response = requests.get(url)
#     return response.text

# def stop_instance(instance_id, region_name='us-east-1'):
#     """Stop the EC2 instance"""
#     ec2 = boto3.client('ec2', region_name=region_name)
#     ec2.stop_instances(InstanceIds=[instance_id])

# # Get the current instance ID
# instance_id = get_instance_id()

# # Stop the instance
# stop_instance(instance_id)